In [ ]:
#=============================================================================================================
"""
This script performs a multi-phase, multi-objective optimization using the Adaptive Weighted Product method. 
The objectives are to maximize the final total mass of the vehicle and the final downrange distance.

Initially, it generates a Pareto front using uniform weights. Then, it refines the results by adaptively 
adjusting the weight distribution to better capture the trade-offs between objectives.

"""
# ============================================================================================================

from pyomo.dae import ContinuousSet, DerivativeVar, Integral
from pyomo.environ import ConcreteModel, TransformationFactory, Var, \
                          NonNegativeReals, Constraint, ConstraintList, \
                          SolverFactory, Objective,  cos, sin, tan, minimize, maximize,  \
                          NonNegativeReals, NegativeReals, Param, sqrt
from pyomo.environ import *
from pyomo.dae import *
from math import isclose
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import logging
from MAV import MAV
from Utilities.Plotter import plotResults
from Utilities.saveOptimizationVariables import saveOptimizationVariables
from Utilities.loadOptimizationVariables import loadOptimizationVariables
from pyomo.environ import Suffix, ConcreteModel, Var, NonNegativeReals, \
    Constraint, Objective, SolverFactory
from pyomo.util.infeasible import (
    find_infeasible_constraints,
    log_infeasible_constraints,
    find_infeasible_bounds,
    log_infeasible_bounds,
    find_close_to_bounds,
    log_close_to_bounds,
)

# Set up logging
logging.basicConfig(level=logging.INFO)  # Ensure logging level is INFO or DEBUG
logger = logging.getLogger('pyomo.core')

def export_to_csv(results_list, filename):
    if not results_list:
        print(f"Warning: Empty results list for {filename}")
        return

    keys = set().union(*(d.keys() for d in results_list))
    
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=list(keys))
        dict_writer.writeheader()
        dict_writer.writerows(results_list)
        
    print(f"Successfully wrote {len(results_list)} points to {filename}")

def select_pareto_points(solutions, trajectories=None, num_points=8):
    
    if len(solutions) <= num_points:
        selected_indices = list(range(len(solutions)))
    else:
        sorted_data = enumerate(solutions)
        sorted_indices = [idx for idx, _ in sorted_data]
        selected_indices = [sorted_indices[0], sorted_indices[-1]]
        if num_points > 2:
            step = (len(sorted_indices) - 1) / (num_points - 1)
            for i in range(1, num_points-1):
                idx = sorted_indices[int(round(i * step))]
                selected_indices.append(idx)
        
        selected_indices = sorted(set(selected_indices))
    colors = ['#0b0b0b', '#0000ff', '#fa9b13', '#2eb82e', '#fa321f', '#00baff', '#ff00e8', '#00ff60']
    color_map = {idx: colors[i % len(colors)] for i, idx in enumerate(selected_indices)}
    
    return selected_indices, color_map

def plot_pareto_front(filtered_solutions, J_U, J_N, mav, num_points=8):
   
    plt.figure(figsize=(10, 6))
    masses = [sol[0] for sol in filtered_solutions]
    ranges = [sol[1] for sol in filtered_solutions]
    ju_jn_masses = [m * (J_U[0] - J_N[0]) + J_N[0] for m in masses]
    ju_jn_ranges = [r * (J_U[1] - J_N[1]) + J_N[1] for r in ranges]
    physical_masses = [m * mav.m.mass_scale for m in ju_jn_masses]
    physical_ranges = [r * mav.m.x_scale/1000 for r in ju_jn_ranges]  # Convert to km
    selected_indices, color_map = select_pareto_points(filtered_solutions, num_points=num_points)
    plt.plot(physical_ranges, physical_masses, 'o-.', color='lightblue', alpha=0.5, markersize=14)
    for idx in selected_indices:
        mass = physical_masses[idx]
        range_val = physical_ranges[idx]
        color = color_map[idx]
        plt.scatter(range_val, mass, color=color, s=220, zorder=5)
   
    plt.xlabel('Downrange [km]', fontsize=20)
    plt.ylabel('Total Vehicle Mass Mass [kg]', fontsize=20)
    plt.grid(True, color='gray', alpha=0.3, linewidth=0.5)
    plt.tick_params(axis='both', which='major', labelsize=20) 
    plt.tight_layout()
    return plt.gcf()

def plot_pareto_trajectories(filtered_solutions, filtered_trajectories, num_trajectories=8):
    selected_indices, color_map = select_pareto_points(
        filtered_solutions, filtered_trajectories, num_trajectories)
    fig1, ax1 = plt.subplots(figsize=(10, 8))  # Time vs Altitude
    fig2, ax2 = plt.subplots(figsize=(10, 8))  # Downrange vs Altitude
    fig3, ax3 = plt.subplots(figsize=(10, 8))  # Downrange vs Altitude
    for idx in selected_indices:
        solution = filtered_solutions[idx]
        trajectory = filtered_trajectories[idx]
        mass, range_val, weights = solution
        color = color_map[idx]
        all_points = (trajectory['phase1'] + trajectory['phase2'] + trajectory['phase3'])
        times = [point['t'] for point in all_points]
        altitudes = [point['z']/1000 for point in all_points]  # Convert to km
        downranges = [np.sqrt(point['x']**2 + point['y']**2)/1000 for point in all_points]  # Convert to km
        mass = [point['mass'] for point in all_points]
        ax1.plot(times, altitudes, color=color, linestyle='--', linewidth=3)
        ax2.plot(downranges, altitudes, color=color, linestyle='--', linewidth=3)
        ax3.plot(times, mass, color=color, linestyle='--', linewidth=3)

        # Plot phase transitions
        if trajectory['phase1']:
            t1_end = trajectory['phase1'][-1]['t']
            alt1_end = trajectory['phase1'][-1]['z']/1000
            downrange1_end = np.sqrt(trajectory['phase1'][-1]['x']**2 + 
                                   trajectory['phase1'][-1]['y']**2)/1000
            mass1_end = trajectory['phase1'][-1]['mass']
            
            ax1.plot(t1_end, alt1_end, color='black', marker='+', markersize=18)
            ax2.plot(downrange1_end, alt1_end, color='black', marker='+', markersize=18)
            ax3.plot(t1_end, mass1_end, color='black', marker='+', markersize=18)
            
        if trajectory['phase2']:
            t2_end = trajectory['phase2'][-1]['t']
            alt2_end = trajectory['phase2'][-1]['z']/1000
            downrange2_end = np.sqrt(trajectory['phase2'][-1]['x']**2 + 
                                   trajectory['phase2'][-1]['y']**2)/1000
            mass2_end = trajectory['phase2'][-1]['mass']
            ax1.plot(t2_end, alt2_end, color='black', marker='x', markersize=18)
            ax2.plot(downrange2_end, alt2_end, color='black', marker='x', markersize=18)
            ax3.plot(t2_end, mass2_end, color='black', marker='x', markersize=18)
            
    ax1.set_xlabel('Time [s]', fontsize=20)
    ax1.set_ylabel('Altitude [km]', fontsize=20)
    ax1.grid(True, color='gray', alpha=0.3, linewidth=0.5)
    ax1.tick_params(axis='both', which='major', labelsize=20)
    fig1.tight_layout()

    ax2.set_xlabel('Downrange [km]', fontsize=20)
    ax2.set_ylabel('Altitude [km]', fontsize=20)
    ax2.grid(True, color='gray', alpha=0.3, linewidth=0.5)
    ax2.tick_params(axis='both', which='major', labelsize=20)
    fig2.tight_layout()

    ax3.set_xlabel('Time [s]', fontsize=20)
    ax3.set_ylabel('Total Vehicle Mass [kg]', fontsize=20)
    ax3.grid(True, color='gray', alpha=0.3, linewidth=0.5)
    ax3.tick_params(axis='both', which='major', labelsize=20)
    fig3.tight_layout()

    end_point_legend = [
        mlines.Line2D([], [], color='black', marker='+', linestyle='None', markersize=10, label='Phase 1 End'),
        mlines.Line2D([], [], color='black', marker='x', linestyle='None', markersize=10, label='Phase 2 End')
    ]

    for ax in [ax1, ax2, ax3]:
        if ax == ax3:
            leg = ax.legend(handles=end_point_legend, title='Phases', title_fontsize=20, fontsize=20, loc='upper right')
        else:
            leg = ax.legend(handles=end_point_legend, title='Phases', title_fontsize=20, fontsize=20, loc='center right')
        leg.get_title().set_fontweight('bold')    

        ax.relim()
        ax.autoscale()
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xlim(left=0, right=xlim[1])
        ax.set_ylim(bottom=0, top=ylim[1])
    return fig1, fig2, fig3

def store_trajectory_data(mav):

    trajectory = {
        'phase1': [],
        'phase2': [],
        'phase3': []
    }
    
    # Store Phase 1 data
    for t1 in mav.m.t1:
        point = {
            't': t1 * value(mav.m.tf1),
            'x': value(mav.m.x_1[t1] * mav.m.x_scale),
            'y': value(mav.m.y_1[t1] * mav.m.y_scale),
            'downrange': sqrt(value(mav.m.x_1[t1] * mav.m.x_scale)**2 + value(mav.m.y_1[t1] * mav.m.y_scale)**2),
            'z': value(mav.m.z_1[t1] * mav.m.z_scale),
            'q0': value(mav.m.q0_1[t1] * mav.m.q0_scale),
            'q1': value(mav.m.q1_1[t1] * mav.m.q1_scale),
            'q2': value(mav.m.q2_1[t1] * mav.m.q2_scale),
            'q3': value(mav.m.q3_1[t1] * mav.m.q3_scale),
            'eps': np.rad2deg(value(mav.m.eps_1[t1] * mav.m.eps_scale)),
            'kap': np.rad2deg(value(mav.m.kap_1[t1] * mav.m.kap_scale)),
            'mass': value(mav.m.mass_1[t1] * mav.m.mass_scale),
            'mpdot': value(mav.m.mpdot_1[t1] * mav.m.mpdot_scale),
            'p': value(mav.m.p_1[t1] * mav.m.p_scale),
            'q': value(mav.m.q_1[t1] * mav.m.q_scale),
            'r': value(mav.m.r_1[t1] * mav.m.r_scale),
            'u': value(mav.m.u_1[t1] * mav.m.u_scale),
            'v': value(mav.m.v_1[t1] * mav.m.v_scale),
            'w': value(mav.m.w_1[t1] * mav.m.w_scale),
        }
        trajectory['phase1'].append(point)

    # Store Phase 2 data
    for t2 in mav.m.t2:
        point = {
            't': t2 * value(mav.m.tf2) + value(mav.m.tf1()),
            'x': value(mav.m.x_2[t2] * mav.m.x_scale),
            'y': value(mav.m.y_2[t2] * mav.m.y_scale),
            'downrange': sqrt(value(mav.m.x_2[t2] * mav.m.x_scale)**2 + value(mav.m.y_2[t2] * mav.m.y_scale)**2),
            'z': value(mav.m.z_2[t2] * mav.m.z_scale),
            'q0': value(mav.m.q0_2[t2] * mav.m.q0_scale),
            'q1': value(mav.m.q1_2[t2] * mav.m.q1_scale),
            'q2': value(mav.m.q2_2[t2] * mav.m.q2_scale),
            'q3': value(mav.m.q3_2[t2] * mav.m.q3_scale),
            'eps': np.rad2deg(value(mav.m.eps_2[t2] * mav.m.eps_scale)),
            'kap': np.rad2deg(value(mav.m.kap_2[t2] * mav.m.kap_scale)),
            'mass': value(mav.m.mass_2[t2] * mav.m.mass_scale),
            'mpdot': value(mav.m.mpdot_2[t2] * mav.m.mpdot_scale),
            'p': value(mav.m.p_2[t2] * mav.m.p_scale),
            'q': value(mav.m.q_2[t2] * mav.m.q_scale),
            'r': value(mav.m.r_2[t2] * mav.m.r_scale),
            'u': value(mav.m.u_2[t2] * mav.m.u_scale),
            'v': value(mav.m.v_2[t2] * mav.m.v_scale),
            'w': value(mav.m.w_2[t2] * mav.m.w_scale),
        }
        trajectory['phase2'].append(point)
    
    # Store Phase 3 data
    for t3 in mav.m.t3:
        point = {
            't': t3 * value(mav.m.tf3) + mav.m.tf1() + mav.m.tf2(),
            'x': value(mav.m.x_3[t3] * mav.m.x_scale),
            'y': value(mav.m.y_3[t3] * mav.m.y_scale),
            'downrange': sqrt(value(mav.m.x_3[t3] * mav.m.x_scale)**2 + value(mav.m.y_3[t3] * mav.m.y_scale)**2),
            'z': value(mav.m.z_3[t3] * mav.m.z_scale),
            'q0': value(mav.m.q0_3[t3] * mav.m.q0_scale),
            'q1': value(mav.m.q1_3[t3] * mav.m.q1_scale),
            'q2': value(mav.m.q2_3[t3] * mav.m.q2_scale),
            'q3': value(mav.m.q3_3[t3] * mav.m.q3_scale),
            'eps': np.rad2deg(value(mav.m.eps_3[t3] * mav.m.eps_scale)),
            'kap': np.rad2deg(value(mav.m.kap_3[t3] * mav.m.kap_scale)),
            'mass': value(mav.m.mass_3[t3] * mav.m.mass_scale),
            'mpdot': value(mav.m.mpdot_3[t3] * mav.m.mpdot_scale),
            'p': value(mav.m.p_3[t3] * mav.m.p_scale),
            'q': value(mav.m.q_3[t3] * mav.m.q_scale),
            'r': value(mav.m.r_3[t3] * mav.m.r_scale),
            'u': value(mav.m.u_3[t3] * mav.m.u_scale),
            'v': value(mav.m.v_3[t3] * mav.m.v_scale),
            'w': value(mav.m.w_3[t3] * mav.m.w_scale),
        }
        trajectory['phase3'].append(point)
    return trajectory

def adaptive_weighted_sum_optimization(mav, max_iterations=7):
   
    mass_opt, range_at_mass_opt = solve_single_objective(mav, 'mass')
    mass_at_range_opt, range_opt = solve_single_objective(mav, 'range')
    J_U = [mass_opt , range_opt]
    J_N = [mass_at_range_opt , range_at_mass_opt] 
    solutions, trajectories = run_initial_optimization(mav, J_U, J_N)
    needs_refinement, segments = check_convergence_and_lengths(solutions, trajectories)
    iteration = 1
    print(f"\nIteration {iteration}")

    while True:
        if not needs_refinement or iteration >= max_iterations:
            break
        print(f"\nIteration {iteration + 1}")
        
        new_solutions = []
        new_trajectories = []

        for segment in segments:
            sub_solutions, sub_trajectories = sub_optimize_segment(mav, segment, J_U, J_N)
            if sub_solutions:
                new_solutions.extend(sub_solutions)
                new_trajectories.extend(sub_trajectories)

        all_solutions = solutions + new_solutions
        all_trajectories = trajectories + new_trajectories
        solutions, trajectories = filter_solutions(all_solutions, all_trajectories)
        needs_refinement, segments = check_convergence_and_lengths(solutions, trajectories)
        iteration += 1
    
    return solutions, trajectories, J_U, J_N

# Step 1 calculate the single objective optimal points as well as the normals
def solve_single_objective(mav, objective_type='mass'):
    if hasattr(mav.m, 'objective'):
        mav.m.del_component('objective')
    if objective_type == 'mass':
        mav.m.objective = Objective(expr=mav.m.mass, sense=maximize)
    else:  # range
        mav.m.objective = Objective(expr=mav.m.range, sense=maximize)

    # Solve
    solver = SolverFactory('ipopt')
    solver.options["halt_on_ampl_error"] = "yes"
    solver.options['tol'] = 1e-7
    solver.options['dual_inf_tol'] = 1e-7
    solver.options['constr_viol_tol'] = 1e-7
    solver.options["max_iter"] = 2000
    solver.options["linear_scaling_on_demand"] = "yes"
    solver.options['nlp_scaling_method'] = 'gradient-based'
    solver.options['linear_solver'] = "ma27"
    solver.options["ma27_pivtol"] = 1e-7
    solver.options['acceptable_tol'] = 1e-7
    results = solver.solve(mav.m, tee=True, keepfiles=True, logfile="log_check.log")
    return value(mav.m.mass), value(mav.m.range)

# Step 2 perform multiobjective optimization using the usutal weighted-sum approach
def run_initial_optimization(mav, J_U, J_N):
    final_mass_values = []
    final_downrange_values = []
    n_initial = 5
    delta_lambda = 1.0 / n_initial
    weights = [(i * delta_lambda, 1.0 - i * delta_lambda) 
              for i in range(n_initial + 1)]
    solutions = []
    trajectories = [] 

    mav.m.del_component(mav.m.C_range)
    mav.m.del_component(mav.m.C_mass)
    mav.m.del_component(mav.m.range)
    mav.m.del_component(mav.m.mass)
    
    for w1, w2 in weights:
        
        if hasattr(mav.m, 'objective'):
            mav.m.del_component('objective')
        
        mav.m.range = Var(bounds=(J_N[1] + 0.0005, None))
        mav.m.mass = Var(bounds=(J_N[0] + 0.0005, None))
        mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_3[1]))
        mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_3[1]))
        normalized_mass = ((mav.m.mass) - J_N[0]) / (J_U[0] - J_N[0])
        normalized_range = ((mav.m.range) - J_N[1]) / (J_U[1] - J_N[1])
        
        mav.m.objective = Objective(expr=(normalized_mass**w1 * normalized_range**w2), sense=maximize
        )

        # Solve
        solver = SolverFactory('ipopt')
        solver.options["halt_on_ampl_error"] = "yes"
        solver.options['tol'] = 1e-7
        solver.options['dual_inf_tol'] = 1e-7
        solver.options['constr_viol_tol'] = 1e-7
        solver.options["max_iter"] = 1200
        solver.options["linear_scaling_on_demand"] = "yes"
        solver.options['nlp_scaling_method'] = 'gradient-based'
        solver.options['linear_solver'] = "ma27"
        solver.options["ma27_pivtol"] = 1e-7
        solver.options['mu_init'] = 1e-1 
        solver.options['acceptable_tol'] = 1e-7

        results = solver.solve(mav.m, tee=True)
        if (results.solver.status == SolverStatus.ok and 
        results.solver.termination_condition == TerminationCondition.optimal):
            solution =  (value(normalized_mass), value(normalized_range), (w1, w2))
        if solution:
            solutions.append(solution)
            trajectory = store_trajectory_data(mav)
            
            trajectories.append(trajectory)
            final_mass_values.append(value(normalized_mass))
            final_downrange_values.append(np.sqrt(value(normalized_range)**2) / 1e3)

        mav.m.del_component(mav.m.C_range)
        mav.m.del_component(mav.m.range)
        mav.m.del_component(mav.m.mass)
        mav.m.del_component(mav.m.C_mass)

    labels = [f'$W_r={w2},W_m={w1}$' for w1, w2 in weights]

    markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'H']
    plt.figure()

    for i in range(len(final_mass_values)):
        plt.scatter(final_mass_values[i], final_downrange_values[i], marker=markers[i % len(markers)], color='black', label=f'Point {i+1}')
        if i == 0:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(-40, 10), ha='center')
        elif i == 1:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(-50, 0), ha='center')
        elif i == 2:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(50, -10), ha='center')
        elif i == 3:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(30, -13), ha='center')
        elif i == 4:
            plt.annotate(labels[i % len(labels)], (final_mass_values[i], final_downrange_values[i]), textcoords="offset points", xytext=(40, 3), ha='center')

    plt.xlabel('Vehicle Mass[km]')
    plt.ylabel('Total final_downrange_values [kg]')
    plt.legend()
    plt.grid(True)
    plt.show()

    filtered_solutions, filtered_trajectories = filter_solutions(solutions, trajectories)

    return filtered_solutions, filtered_trajectories

# Step 3 compute the length between neighboring solutions and delete overlapping solutions (euclidean distance < epsilon)
def filter_solutions(solutions, trajectories, epsilon=0.025):
    filtered_solutions = []
    filtered_trajectories = []
    
    flat_solutions = []
    flat_trajectories = []
    
    for i, sol in enumerate(solutions):
        if isinstance(sol, list):
            flat_solutions.extend(sol)
            flat_trajectories.extend([trajectories[i]] * len(sol))
        elif isinstance(sol, tuple) and len(sol) == 3:
            mass, range_val, weight = sol
            if not isinstance(weight, tuple):
                weight = (weight, 1.0 - weight)
            flat_solutions.append((mass, range_val, weight))
            flat_trajectories.append(trajectories[i])

    for i, sol1 in enumerate(flat_solutions):
        is_unique = True
        for sol2 in filtered_solutions:
            dist = calculate_euclidean_distance(
                (sol1[0], sol1[1]),
                (sol2[0], sol2[1])
            )
            if dist < epsilon:
                is_unique = False
                break
        if is_unique:
            filtered_solutions.append(sol1)
            filtered_trajectories.append(flat_trajectories[i])
    
    return filtered_solutions, filtered_trajectories

def calculate_euclidean_distance(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Step 4 Determine the number of further refinements in each of the regions
def check_convergence_and_lengths(solutions, trajectories, max_length=0.05, C=2.0):

    if len(solutions) < 2:
        return False, []
    
    indices = list(range(len(solutions)))
    indices = [x for _, x in sorted(zip(solutions, indices), key=lambda pair: pair[0][1])]
    solutions.sort(key=lambda x: x[1])
    trajectories[:] = [trajectories[i] for i in indices]
    lengths, avg_length = calculate_segment_lengths(solutions)
   
    needs_refinement = False
    segments_to_refine = []
  
    for i, length in enumerate(lengths):
        if length > max_length:
            needs_refinement = True
            n_i = round(C * length / avg_length)
            if n_i > 1:
                point1 = (solutions[i][0], solutions[i][1])
                point2 = (solutions[i+1][0], solutions[i+1][1])
                delta1, delta2 = calculate_segment_offsets(point1, point2)
                segments_to_refine.append({
                    'index': i,
                    'start_point': solutions[i],
                    'end_point': solutions[i + 1],
                    'length': length,
                    'refinements': n_i,
                    'offsets': (delta1, delta2)
                })
    return needs_refinement, segments_to_refine

# Step 5 Calculate the length segment length to determine if ni < 1
def calculate_segment_lengths(solutions):
   
    lengths = []
    for i in range(len(solutions) - 1):
        point1 = (solutions[i][0], solutions[i][1])
        point2 = (solutions[i + 1][0], solutions[i + 1][1])
        length = calculate_euclidean_distance(point1, point2)
        lengths.append(length)
    
    avg_length = sum(lengths) / len(lengths) if lengths else 0
    return lengths, avg_length

# Step 6 Determine offset distances from the two end points of each segment
def calculate_segment_offsets(point1, point2, delta_f=0.05):
    
    P1x, P1y = point1
    P2x, P2y = point2
    theta = np.arctan2(-(P2y - P1y), (P2x - P1x))
    delta1 = delta_f * np.cos(theta)
    delta2 = delta_f * np.sin(theta)
    
    return delta1, delta2

def determine_refinements(lengths, avg_length, C=2.0):

    refinements = []
    for length in lengths:
        n_i = round(C * length / avg_length)
        refinements.append(max(0, n_i))  
    return refinements

# Step 7 Impose additional inequality constraints and conduct sub-optimization
def sub_optimize_segment(mav, segment, J_U, J_N):
   
    start_point = segment['start_point']
    end_point = segment['end_point']
    delta1, delta2 = segment['offsets']
    n_i = segment['refinements']
    
    delta_lambda = 1.0 / n_i
    sub_solutions = []
    sub_trajectories = []
    
    lambda_values = np.arange(0, 1 + delta_lambda, delta_lambda)
 
    for lambda_i in lambda_values:

        if hasattr(mav.m, 'objective'):
            mav.m.del_component('objective')
        
        mav.m.range = Var(bounds=(J_N[1] + 0.0005, None))
        mav.m.mass = Var(bounds=(J_N[0] + 0.0005, None))
        mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_3[1]))
        mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_3[1]))
        normalized_mass = ((mav.m.mass) - J_N[0]) / (J_U[0] - J_N[0])
        normalized_range = ((mav.m.range) - J_N[1]) / (J_U[1] - J_N[1])

        mav.m.ineq_mass = Constraint(
            expr=normalized_mass <= start_point[0] - delta1
        )
        mav.m.ineq_range = Constraint(
            expr=normalized_range >= end_point[1] - delta2
        )
        
        mav.m.objective = Objective(
            expr=((normalized_mass**lambda_i)  * (normalized_range ** (1 - lambda_i)) ),
            sense=maximize
        )
        
        # Solve
        solver = SolverFactory('ipopt')
        solver.options["halt_on_ampl_error"] = "yes"
        solver.options['tol'] = 1e-7
        solver.options['dual_inf_tol'] = 1e-7
        solver.options['constr_viol_tol'] = 1e-7
        solver.options["max_iter"] = 1200
        solver.options["linear_scaling_on_demand"] = "yes"
        solver.options['nlp_scaling_method'] = 'gradient-based'
        solver.options['linear_solver'] = "ma27"
        solver.options["ma27_pivtol"] = 1e-7
        solver.options['max_cpu_time'] = 120
        solver.options['acceptable_tol'] = 1e-7
        results = solver.solve(mav.m, tee=False)
        
        if (results.solver.status == SolverStatus.ok and 
        results.solver.termination_condition == TerminationCondition.optimal):
            sol = (value(normalized_mass), value(normalized_range), lambda_i)
            traj = store_trajectory_data(mav)
            sub_solutions.append(sol)
            sub_trajectories.append(traj)

        mav.m.del_component(mav.m.ineq_mass)
        mav.m.del_component(mav.m.ineq_range)
        mav.m.del_component(mav.m.C_range)
        mav.m.del_component(mav.m.range)
        mav.m.del_component(mav.m.mass)
        mav.m.del_component(mav.m.C_mass)
    
    return sub_solutions, sub_trajectories

def main():

    # Weightings 
    W_Obj1 = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    W_Obj2 = [0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0] 

    for weight in range(1):
        for warm in range(1):

            # Create mav vehicle
            mav = MAV()
        
            # Boundary Conditions
            mav.m.BCs_con = ConstraintList(rule=mav.BCs)
            
            # Phase 1
            n1 = [1] # flag for Phase 1

            mav.m.Q_dmpdot_dtau_Con1         = Constraint(n1, mav.m.t1, rule=mav.Q_dmass_dtau)

            mav.m.Q_dx_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dx_dtau)
            mav.m.Q_dy_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dy_dtau)
            mav.m.Q_dz_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dz_dtau)

            mav.m.Q_du_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_du_dtau)
            mav.m.Q_dv_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dv_dtau)
            mav.m.Q_dw_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dw_dtau)

            mav.m.Q_dp_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dp_dtau)
            mav.m.Q_dq_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dq_dtau)
            mav.m.Q_dr_dtau_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_dr_dtau)

            mav.m.Q_dq0_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dq0_dtau)
            mav.m.Q_dq1_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dq1_dtau)
            mav.m.Q_dq2_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dq2_dtau)
            mav.m.Q_dq3_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dq3_dtau)

            # mav.m.Q_dphi_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dphi_dtau)
            # mav.m.Q_dthe_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dthe_dtau)
            # mav.m.Q_dpsi_dtau_Con1           = Constraint(n1, mav.m.t1, rule=mav.Q_dpsi_dtau)

            mav.m.Q_massdot_Con1             = Constraint(n1, mav.m.t1, rule=mav.Q_massdot)

            # mav.m.Q_q0_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_q0)
            # mav.m.Q_q1_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_q1)
            # mav.m.Q_q2_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_q2)
            # mav.m.Q_q3_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_q3)'

            mav.m.Q_pdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_pdot)
            mav.m.Q_qdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_qdot)   
            mav.m.Q_rdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_rdot)

            mav.m.Q_udot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_udot)
            mav.m.Q_vdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_vdot)   
            mav.m.Q_wdot_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_wdot)

            mav.m.Q_q0dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q0dot)
            mav.m.Q_q1dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q1dot)
            mav.m.Q_q2dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q2dot)
            mav.m.Q_q3dot_Con1               = Constraint(n1, mav.m.t1, rule=mav.Q_q3dot)

            # mav.m.Q_phidot_Con1              = Constraint(n1, mav.m.t1, rule=mav.Q_phidot)
            # mav.m.Q_thedot_Con1              = Constraint(n1, mav.m.t1, rule=mav.Q_thedot)
            # mav.m.Q_psidot_Con1              = Constraint(n1, mav.m.t1, rule=mav.Q_psidot)

            mav.m.Q_u1_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_u)
            mav.m.Q_v1_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_v)
            mav.m.Q_w1_Con1                  = Constraint(n1, mav.m.t1, rule=mav.Q_w)

            mav.m.Q_normality_Con1                = Constraint(n1, mav.m.t1, rule=mav.Q_normality)

            # Phase 2
            n2 = [2] # flag for Phase 2

            mav.m.Q_dmpdot_dtau_Con2         = Constraint(n2, mav.m.t2, rule=mav.Q_dmass_dtau)

            mav.m.Q_massdot2_Con2            = Constraint(n2, mav.m.t2, rule=mav.Q_mass_dot_2)

            mav.m.Q_dx_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dx_dtau)
            mav.m.Q_dy_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dy_dtau)
            mav.m.Q_dz_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dz_dtau)

            mav.m.Q_du_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_du_dtau)
            mav.m.Q_dv_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dv_dtau)
            mav.m.Q_dw_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dw_dtau)

            mav.m.Q_dp_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dp_dtau)
            mav.m.Q_dq_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dq_dtau)
            mav.m.Q_dr_dtau_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_dr_dtau)

            mav.m.Q_dq0_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dq0_dtau)
            mav.m.Q_dq1_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dq1_dtau)
            mav.m.Q_dq2_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dq2_dtau)
            mav.m.Q_dq3_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dq3_dtau)

            # mav.m.Q_dphi_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dphi_dtau)
            # mav.m.Q_dthe_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dthe_dtau)
            # mav.m.Q_dpsi_dtau_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_dpsi_dtau)

            mav.m.Q_massdot_Con2             = Constraint(n2, mav.m.t2, rule=mav.Q_massdot)

            # mav.m.Q_q0_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_q0)
            # mav.m.Q_q1_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_q1)
            # mav.m.Q_q2_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_q2)
            # mav.m.Q_q3_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_q3)

            mav.m.Q_pdot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_pdot)
            mav.m.Q_qdot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_qdot_2)   
            mav.m.Q_rdot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_rdot_2)
            
            mav.m.Q_udot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_udot_2)
            mav.m.Q_vdot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_vdot_2)   
            mav.m.Q_wdot_Con2                = Constraint(n2, mav.m.t2, rule=mav.Q_wdot_2)

            # mav.m.Q_phidot_Con2              = Constraint(n2, mav.m.t2, rule=mav.Q_phidot)
            # mav.m.Q_thedot_Con2              = Constraint(n2, mav.m.t2, rule=mav.Q_thedot)
            # mav.m.Q_psidot_Con2              = Constraint(n2, mav.m.t2, rule=mav.Q_psidot)

            mav.m.Q_q0dot_Con2               = Constraint(n2, mav.m.t2, rule=mav.Q_q0dot)
            mav.m.Q_q1dot_Con2               = Constraint(n2, mav.m.t2, rule=mav.Q_q1dot)
            mav.m.Q_q2dot_Con2               = Constraint(n2, mav.m.t2, rule=mav.Q_q2dot)
            mav.m.Q_q3dot_Con2               = Constraint(n2, mav.m.t2, rule=mav.Q_q3dot)

            mav.m.Q_u1_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_u)
            mav.m.Q_v1_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_v)
            mav.m.Q_w1_Con2                  = Constraint(n2, mav.m.t2, rule=mav.Q_w)

            mav.m.Q_normality_Con2           = Constraint(n2, mav.m.t2, rule=mav.Q_normality)

            # Phase 3
            n3 = [3] # flag for Phase 3

            mav.m.Q_dmpdot_dtau_Con3         = Constraint(n3, mav.m.t3, rule=mav.Q_dmass_dtau)

            mav.m.Q_dx_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dx_dtau)
            mav.m.Q_dy_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dy_dtau)
            mav.m.Q_dz_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dz_dtau)

            mav.m.Q_du_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_du_dtau)
            mav.m.Q_dv_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dv_dtau)
            mav.m.Q_dw_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dw_dtau)

            mav.m.Q_dp_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dp_dtau)
            mav.m.Q_dq_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dq_dtau)
            mav.m.Q_dr_dtau_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_dr_dtau)

            mav.m.Q_dq0_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dq0_dtau)
            mav.m.Q_dq1_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dq1_dtau)
            mav.m.Q_dq2_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dq2_dtau)
            mav.m.Q_dq3_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dq3_dtau)

            # mav.m.Q_dphi_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dphi_dtau)
            # mav.m.Q_dthe_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dthe_dtau)
            # mav.m.Q_dpsi_dtau_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_dpsi_dtau)

            mav.m.Q_massdot_Con3             = Constraint(n3, mav.m.t3, rule=mav.Q_massdot)

            # mav.m.Q_q0_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_q0)
            # mav.m.Q_q1_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_q1)
            # mav.m.Q_q2_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_q2)
            # mav.m.Q_q3_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_q3)

            mav.m.Q_pdot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_pdot)
            mav.m.Q_qdot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_qdot)   
            mav.m.Q_rdot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_rdot)
            
            mav.m.Q_udot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_udot )
            mav.m.Q_vdot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_vdot)   
            mav.m.Q_wdot_Con3                = Constraint(n3, mav.m.t3, rule=mav.Q_wdot)

            mav.m.Q_q0dot_Con3               = Constraint(n3, mav.m.t3, rule=mav.Q_q0dot)
            mav.m.Q_q1dot_Con3               = Constraint(n3, mav.m.t3, rule=mav.Q_q1dot)
            mav.m.Q_q2dot_Con3               = Constraint(n3, mav.m.t3, rule=mav.Q_q2dot)
            mav.m.Q_q3dot_Con3               = Constraint(n3, mav.m.t3, rule=mav.Q_q3dot)

            # mav.m.Q_phidot_Con3              = Constraint(n3, mav.m.t3, rule=mav.Q_phidot)
            # mav.m.Q_thedot_Con3              = Constraint(n3, mav.m.t3, rule=mav.Q_thedot)
            # mav.m.Q_psidot_Con3              = Constraint(n3, mav.m.t3, rule=mav.Q_psidot)

            mav.m.Q_u1_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_u)
            mav.m.Q_v1_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_v)
            mav.m.Q_w1_Con3                  = Constraint(n3, mav.m.t3, rule=mav.Q_w)

            mav.m.Q_normality_Con3           = Constraint(n3, mav.m.t3, rule=mav.Q_normality)

            mav.m.range = Var(bounds=(0.01, None))
            mav.m.mass = Var(bounds=(0.01, None))
            
            mav.m.C_range = Constraint(expr=mav.m.range == (mav.m.x_3[1]))
            mav.m.C_mass = Constraint(expr=mav.m.mass == (mav.m.mass_3[1]))

            if weight == 0:
                filtered_solutions, filtered_trajectories, J_U, J_N = adaptive_weighted_sum_optimization(mav)
                if not filtered_solutions:
                    print("No solutions found!")
                    return None

                for i, (solution, trajectory) in enumerate(zip(filtered_solutions, filtered_trajectories)):
                    mass, range_val, weights = solution
                    print(f"\nPareto Point {i+1}:")
                    print(f"Mass: {mass:.6f}")
                    print(f"Range: {range_val:.6f}")
                    print(f"Weights: {weights}")
                    results_list = trajectory['phase1'] + trajectory['phase2'] + trajectory['phase3']
                    filename = f'pareto_trajectory_{i+1}.csv'
                    export_to_csv(results_list, filename)
                plot_pareto_front(filtered_solutions, J_U, J_N, mav, num_points=8)
                plot_pareto_trajectories(filtered_solutions, filtered_trajectories, num_trajectories=8) 

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
       
    return mav.m

if __name__ == '__main__':
    m = main()

    